In [50]:
import pandas as pd

# Librerías usadas para estandarizar la información
import dicts 
import utils

#Librerías gráficas y de procesamiento matemático
import matplotlib.pyplot as plt
import seaborn as sns
#import plotly.express as px
import numpy as np

## Lectura de datos poblacionales con el propósito de identificar municipios con población  femenina cero que deben ser excluidos del análisis

In [51]:
df1=pd.read_csv("anexo-area-sexo-edad-proyecciones-poblacion-Municipal_1995-2004.csv")
df2=pd.read_csv("anexo-area-sexo-edad-proyecciones-poblacion-Municipal_2005-2017.csv")

### Filtro x población

Los municipios cuya población entre 1998 y 2004 fue cero se enumeran

In [52]:
df1=df1[df1["AÑO"]>=1998]
df1["Total Mujeres"]=sum(df1[f"Mujeres_{i}"] for i in range(101))
pob_0=set(df1[df1["Total Mujeres"]==0].DPMP.unique())
list(df1[df1.DPMP.isin(pob_0)].MPIO.unique())

['Norosí',
 'Norcasia',
 'Guachené',
 'Sucre',
 'Villa Rica',
 'San José de Uré',
 'Tuchín',
 'Carmen del Darién',
 'Cértegui',
 'Medio Atrato',
 'Medio Baudó',
 'Medio San Juan',
 'Río Iró',
 'Río Quito',
 'Unión Panamericana',
 'Albania',
 'La Jagua del Pilar',
 'Algarrobo',
 'Concordia',
 'Nueva Granada',
 'Sabanas de San Ángel',
 'Santa Bárbara de Pinto',
 'Zapayán',
 'Zona Bananera',
 'El Peñol',
 'Nariño',
 'Coveñas']

In [53]:
len(df1[df1.DPMP.isin(pob_0)].MPIO.unique())

27

In [54]:
df1[df1.DPMP.isin(pob_0)].DPMP.astype(str).unique()

array(['13490', '17495', '19300', '19785', '19845', '23682', '23815',
       '27150', '27160', '27425', '27430', '27450', '27580', '27600',
       '27810', '44035', '44420', '47030', '47205', '47460', '47660',
       '47720', '47960', '47980', '52254', '52480', '70221'], dtype=object)

Un total de 47 municipios tenían una población feménina de cero entre 1998 y 2004. Laz razones de esto puede ser que, si bien aparece en la base de datos, el municipio no se había constituido oficialmente para ese periodo o simplemente que corresponde a un territorio no municipalizado, por ejemplo forestal, donde no habitan personas.

In [55]:
df2["Total Mujeres"]=sum(df2[f"Mujeres_{i}"] for i in range(101))
pob_1=set(df2[df2["Total Mujeres"]==0].DPMP.unique())
list(df2[df2.DPMP.isin(pob_1)].MPIO.unique())

['Norosí', 'Guachené', 'San José de Uré', 'Tuchín']

In [56]:
len(df2[df2.DPMP.isin(pob_1)].MPIO.unique())

4

## Microdatos mortalidad por cáncer de seno

Inicialmente se carga la base de datos

In [57]:
df=pd.DataFrame(pd.read_csv("Cancer_1998_2018.csv"))
#df["codmunicipio"]=df.apply(lambda x: int(str(x.codptore).zfill(2)+str(x.codmunre).zfill(3)),axis=1)
df.to_csv("Mortalidad_1998_2018.csv")
df

,Unnamed: 0,ano,codptore,codmunre,gru_ed1,gru_ed2,area_res
0,0,1998.0,19,19075,14,4,2
1,1,1998.0,27,27001,23,6,1
2,2,1998.0,47,47541,18,5,2
3,3,1998.0,50,50001,22,6,1
4,4,1998.0,54,54099,22,6,3
...,...,...,...,...,...,...,...
46747,46747,2018.0,8,8001,14,4,1
46748,46748,2018.0,11,11001,23,6,1
46749,46749,2018.0,76,76275,20,5,1
46750,46750,2018.0,8,8001,19,5,1


In [44]:
df[df["codmunre"]!=999].to_excel("Cancer_1998_2018.xlsx")

Se eliminan de la tabla los datos correspondientes a los municipios que no se consideran en el análisis

df=df[~df.codmunicipio.isin(pob_0)].copy()
df.head(20)

## Consolidación de los resultados a nivel municipio/año para el periodo de estudio

### exclusión de los datos que no tienen información sobre el municipio

In [58]:
df_try=df.copy()#[df.codmunicipio<99999].copy()
len(df_try.codmunre.unique())
df_try

,Unnamed: 0,ano,codptore,codmunre,gru_ed1,gru_ed2,area_res
0,0,1998.0,19,19075,14,4,2
1,1,1998.0,27,27001,23,6,1
2,2,1998.0,47,47541,18,5,2
3,3,1998.0,50,50001,22,6,1
4,4,1998.0,54,54099,22,6,3
...,...,...,...,...,...,...,...
46747,46747,2018.0,8,8001,14,4,1
46748,46748,2018.0,11,11001,23,6,1
46749,46749,2018.0,76,76275,20,5,1
46750,46750,2018.0,8,8001,19,5,1


Lectura de nombre de departamentos desde la librería de diccionarios

In [59]:
df_try.ano=df_try.ano.astype(int)

newdict={j:i for i,j in zip(dicts.DPTOS.keys(),dicts.DPTOS.values())}
newdict



{68: 'Santander',
 54: 'Norte de Santander',
 47: 'Magdalena',
 20: 'Cesar',
 81: 'Arauca',
 9: 'Sin información de departamento,',
 13: 'Bolivar',
 63: 'Quindio',
 17: 'Caldas',
 73: 'Tolima',
 11: 'Bogotá',
 66: 'Risaralda',
 25: 'Cundinamarca',
 76: 'Valle del Cauca',
 8: 'Atlántico',
 70: 'Sucre',
 44: 'La guajira',
 88: 'Archipiélago S.A, P. y Sta. C.',
 52: 'Nariño',
 19: 'Cauca',
 999: 'Con residencia en el extranjero.',
 86: 'Putumayo',
 27: 'Choco',
 5: 'Antioquia',
 23: 'Cordoba',
 50: 'Meta',
 99: 'Vichada',
 15: 'Boyaca',
 18: 'Caqueta',
 85: 'Casanare',
 41: 'Huila',
 95: 'Guaviare',
 91: 'Amazonas',
 94: 'Guainía',
 97: 'Vaupés'}

Se genera una tabla cruzada con el código del municipio y el año

In [60]:
df_try=pd.DataFrame(pd.crosstab(df_try.codmunre,df_try.ano)).reset_index()
df_try["codmunicipio"]=df_try.codmunre.apply(lambda x: str(x).zfill(5))
df_try

ano,codmunre,1998,1999,2000,2001,2002,2003,2004,2005,2006,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,codmunicipio
0,5001,117,140,152,121,156,181,166,138,181,...,197,206,213,223,236,223,256,272,243,05001
1,5002,0,0,1,0,0,0,1,2,0,...,1,0,0,2,1,1,4,1,0,05002
2,5004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,1,0,1,0,05004
3,5021,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,05021
4,5030,0,1,2,0,0,1,0,1,1,...,3,4,1,1,2,1,3,3,1,05030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,97511,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,97511
1024,99001,0,2,1,0,0,0,1,1,0,...,1,0,0,0,0,0,1,1,0,99001
1025,99524,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,99524
1026,99624,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,99624


### Utilizando información oficial sobre los nombres de los municipios se constuye uno para asociarlos a los códigos d elos municipios

In [62]:
dictio=pd.DataFrame(pd.read_csv("DIVIPOLA_Municipios.csv"))
dictio.COD=dictio.COD.astype(str)
dictio.COD=dictio.COD.apply(lambda x: x.zfill(5))
print(dictio.columns)
dictio=dictio.set_index('COD').T.to_dict()
for i in dictio.keys():
    dictio[i]=dictio[i]['MUNICIPIO'].lower()



Index(['COD', 'MUNICIPIO'], dtype='object')


In [63]:
df_try["codmunre"]=df_try["codmunre"].apply(lambda x: str(x).zfill(5))


In [64]:
df_try["MUNICIPIO"]=df_try.codmunre.map(dictio)
df_try["codptore"]=df_try["codmunre"].apply(lambda x: x[:2])
df_try["DPTO"]=df_try["codptore"].astype(int).map(newdict)
df_try=df_try.fillna(0)

In [65]:
df_try.head(25)

ano,codmunre,1998,1999,2000,2001,2002,2003,2004,2005,2006,...,2013,2014,2015,2016,2017,2018,codmunicipio,MUNICIPIO,codptore,DPTO
0,05001,117,140,152,121,156,181,166,138,181,...,223,236,223,256,272,243,05001,medellín,05,Antioquia
1,05002,0,0,1,0,0,0,1,2,0,...,2,1,1,4,1,0,05002,abejorral,05,Antioquia
2,05004,0,0,0,0,0,0,0,0,0,...,0,2,1,0,1,0,05004,abriaquí,05,Antioquia
3,05021,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,05021,alejandría,05,Antioquia
4,05030,0,1,2,0,0,1,0,1,1,...,1,2,1,3,3,1,05030,amagá,05,Antioquia
5,05031,1,2,0,1,1,0,0,0,0,...,0,2,4,0,0,2,05031,amalfi,05,Antioquia
6,05034,1,1,0,1,1,6,3,2,0,...,2,4,2,3,3,1,05034,andes,05,Antioquia
7,05036,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,05036,angelópolis,05,Antioquia
8,05038,0,0,0,1,3,0,0,0,0,...,0,1,0,0,0,0,05038,angostura,05,Antioquia
9,05040,0,0,0,0,0,0,1,0,2,...,1,0,0,0,0,0,05040,anorí,05,Antioquia
